In [13]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [10]:
data = pd.read_csv('../../Data/dga-full.csv')

In [11]:
data.sample(5)

,dsrc,domain,length,dicts,entropy,numbers,ngram
8501,cryptolocker,dqapqlgmgctibfj,15,0.533333,3.640224,0,1.278754
36802,nivdort,songpure,8,1.000000,3.000000,0,6.537517
24931,necurs,wooebamvjxumkxhqyc,18,0.388889,3.836592,0,2.363612
15463,gameoverdga,fgvyqd1v8qfl916uziixfj39jg,26,0.000000,3.979098,7,0.000000
45412,alexa,towerrecords,12,1.000000,2.855389,0,17.899801


In [22]:
feature_columns = ['length', 'dicts','entropy','numbers','ngram']
scaled_feature_columns = ['scaled_length', 'scaled_dicts','scaled_entropy','scaled_numbers','scaled_ngram']

In [23]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(data[feature_columns])
scaled_df = pd.DataFrame(scaled_df, columns=scaled_feature_columns)

In [24]:
scaled_df.sample(10)

,scaled_length,scaled_dicts,scaled_entropy,scaled_numbers,scaled_ngram
51823,-0.277145,1.144799,0.540150,-0.498385,2.100328
35043,-0.933453,1.144799,-1.265204,-0.498385,0.253893
14156,1.429253,-1.075955,1.418903,1.210521,-0.680919
20960,1.560515,-0.790038,1.377990,0.640886,-0.089887
45251,-0.277145,0.721798,-0.090780,-0.498385,2.997810
14937,1.691776,-1.015526,1.559517,1.210521,-0.722434
47491,-0.670930,1.144799,-0.537386,-0.498385,0.417293
19557,1.429253,-1.287455,1.867059,1.020643,-0.811719
46337,-1.327237,1.144799,-2.262367,-0.498385,-0.179989
9137,-0.014622,0.228297,0.241666,-0.498385,-0.034064


In [27]:
final_data = pd.merge( data, scaled_df, left_index=True, right_index=True)

In [33]:
kmeans = KMeans(n_clusters=2).fit(final_data[scaled_feature_columns])

centroids = kmeans.cluster_centers_
labels = kmeans.labels_

print(centroids)
print(labels)

[[ 1.56899145 -1.17177795  1.22510555  1.5429584  -0.63406745]
 [-0.49760996  0.3716326  -0.3885456  -0.4893535   0.20109624]]
[0 0 0 ..., 1 1 1]


In [34]:
final_data.dsrc.value_counts()

gameoverdga     10000
alexa           10000
necurs          10000
cryptolocker    10000
nivdort          9978
bamital          2500
Name: dsrc, dtype: int64

In [40]:
final_data['cluster_id'] = kmeans.labels_

In [41]:
final_data['cluster_id'].value_counts()

1    39842
0    12636
Name: cluster_id, dtype: int64

In [47]:
final_data[final_data['cluster_id'] == 1]['dsrc'].value_counts()

cryptolocker    10000
alexa            9997
nivdort          9978
necurs           9860
gameoverdga         7
Name: dsrc, dtype: int64

In [48]:
final_data[final_data['cluster_id'] == 0]['dsrc'].value_counts()

gameoverdga    9993
bamital        2500
necurs          140
alexa             3
Name: dsrc, dtype: int64